# Climatología de Temperatura Observada

## Importar librerias

In [1]:
import glob
import pandas as pd
import seaborn as sns
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import copy
from datetime import datetime

## Abrir todos los archivos excel por cuenca

In [2]:
# Criterios de coincidencia de patrones para archivos de Excel en el directorio
filenames = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Temperatura/Cuencas/Cuenca*.xlsx'

In [3]:
# Crea un diccionario vacío para almacenar los datos de cada archivo
dat = {}

# Recorra cada archivo de Excel que coincida con el patrón y lea sus datos en un marco de datos de pandas
for file in glob.glob(filenames):
    dat[file] = pd.read_excel(file)

In [4]:
# Access the data from each file using its file name as the key in the data_dict
data_new = []

for file, data in dat.items():
    data_new.append(data)

## Arreglar datos

* En la columna 'ESTACION' todos los valores tengan 3 digitos

In [5]:
for i, df in enumerate(data_new):
    def formatear_valor(valor):
        if valor < 10:
            return '00' + str(valor)
        elif valor < 100:
            return '0' + str(valor)
        else:
            return str(valor)

    # Aplicar la función a la tabla
    data_new[i]['ESTACION'] = data_new[i]['ESTACION'].apply(formatear_valor)

In [6]:
#Unir el numero de 'CUENCA' con 'ESTACION' para tener el numero de cada estacion
for i, df in enumerate(data_new):
    data_new[i]['ESTACION_NEW'] = data_new[i]['CUENCA'].astype(str) + data_new[i]['ESTACION'].astype(str)

In [9]:
data_new

[      ESTACION_NEW      FECHA    MAX    MIN
 0           100633 2003-10-17  30.37  22.96
 1           100633 2003-10-18  31.28  24.03
 2           100633 2003-10-19  26.45  23.46
 3           100633 2003-10-20  25.54  23.43
 4           100633 2003-10-21  27.59  23.50
 ...            ...        ...    ...    ...
 29327       100655 2022-12-27  32.90  22.70
 29328       100655 2022-12-28  31.50  22.50
 29329       100655 2022-12-29  31.80  22.80
 29330       100655 2022-12-30  32.70  23.10
 29331       100655 2022-12-31  32.60  22.00
 
 [29332 rows x 4 columns],
      ESTACION_NEW      FECHA    MAX    MIN
 0          200002 2002-01-01  28.76  23.30
 1          200002 2002-01-02  28.85  23.93
 2          200002 2002-01-03  26.98  23.89
 3          200002 2002-01-04  28.69  22.63
 4          200002 2002-01-05  28.69  22.87
 ...           ...        ...    ...    ...
 6977       200009 2018-03-04  30.90  21.60
 6978       200009 2018-03-05  32.20  22.30
 6979       200009 2018-03-06  32.5

In [8]:
#Eliminar las columnas que no son necesarias
data_new = [tabla[['ESTACION_NEW', 'FECHA', 'MAX', 'MIN']] for tabla in data_new]

## Agrupar datos por estación

* Agrupar las filas según los valores de la columna "ESTACION_NEW"

In [11]:
data_por_estacion = []

for df in data_new:
    grupos = df.groupby('ESTACION_NEW')
    
    for ESTACION, grupo in grupos:
        data_por_estacion.append(grupo)

* Obtener el número de cada estación

In [14]:
NUMERO = [df.iloc[0, 0] for df in data_por_estacion]

* Separar los datos por temperatura maxima, minima y promedio
* Eliminar las columnas que no son necesarias

In [15]:
TempMAX = [tabla[['FECHA', 'MAX']] for tabla in data_por_estacion]
TempMIN = [tabla[['FECHA', 'MIN']] for tabla in data_por_estacion]

In [18]:
TempMAX

[          FECHA    MAX
 0    2003-10-17  30.37
 1    2003-10-18  31.28
 2    2003-10-19  26.45
 3    2003-10-20  25.54
 4    2003-10-21  27.59
 ...         ...    ...
 6651 2022-12-27  39.10
 6652 2022-12-28  30.00
 6653 2022-12-29  30.90
 6654 2022-12-30  31.00
 6655 2022-12-31  30.20
 
 [6656 rows x 2 columns],
            FECHA    MAX
 6656  2007-03-11  34.71
 6657  2007-03-12  34.91
 6658  2007-03-13  35.18
 6659  2007-03-14  35.05
 6660  2007-03-15  34.71
 ...          ...    ...
 12006 2022-12-27  32.60
 12007 2022-12-28  32.00
 12008 2022-12-29  32.70
 12009 2022-12-30  32.00
 12010 2022-12-31  32.30
 
 [5355 rows x 2 columns],
            FECHA    MAX
 12011 2009-11-25  24.72
 12012 2009-11-26  24.76
 12013 2009-12-13  32.75
 12014 2009-12-14  33.43
 12015 2009-12-15  32.99
 ...          ...    ...
 16771 2022-12-27  31.90
 16772 2022-12-28  31.80
 16773 2022-12-29  32.60
 16774 2022-12-30  31.10
 16775 2022-12-31  32.20
 
 [4765 rows x 2 columns],
            FECHA    MAX
 16

* Filtrar las tablas para que solo tengan valores mayores a -10

In [24]:
# Filtrar y actualizar los DataFrames en TempMAX y TempMIN 
TempMAX = [df.loc[(df['MAX'] > -10) & (df['MAX'] < 50)] for df in TempMAX]
TempMIN = [df.loc[(df['MIN'] > -10) & (df['MIN'] < 50)] for df in TempMIN]

* Volver la columna 'FECHA' un dateintex

In [25]:
# Establecer el índice 'FECHA' en cada DataFrame en TempMAX y TempMIN
TempMAX = [df.set_index('FECHA') for df in TempMAX]
TempMIN = [df.set_index('FECHA') for df in TempMIN]

* Eliminar los valores Nan

In [26]:
# Filtrar y actualizar los DataFrames en TempMAX y TempMIN
TempMAX = [df.dropna(subset=['MAX']) for df in TempMAX]
TempMIN = [df.dropna(subset=['MIN']) for df in TempMIN]

## Obtener los estadisticos diarios

* Crear una lista de las temperaturas Maxima, Minima y Promedio

In [27]:
#Crear una lista vacía llamada lista_de_listas_de_tablas
lista_temp = [TempMAX, TempMIN]

* Obtener los estadisticos mean, median, max y min, de cada T(max, min) y cada estación

In [28]:
# Crear listas para almacenar los resultados de cada estadístico
diario_min = []
diario_max = []
diario_mean = []
diario_median = []
diario_percentil5 = []
diario_percentil95 = []

# Iterar sobre cada lista de tablas en lista_temp
for lista in lista_temp:
    # Crear listas temporales para almacenar los resultados de cada estadístico en la lista actual
    temp_min = []
    temp_max = []
    temp_mean = []
    temp_median = []
    temp_percentil5 = []
    temp_percentil95 = []

    # Iterar sobre cada tabla en la lista actual
    for tabla in lista:
        # Agrupar por mes y día y calcular los estadísticos
        estadisticas_min = tabla.groupby(tabla.index.strftime("%m%d")).min()
        temp_min.append(estadisticas_min)

        estadisticas_max = tabla.groupby(tabla.index.strftime("%m%d")).max()
        temp_max.append(estadisticas_max)

        estadisticas_mean = tabla.groupby(tabla.index.strftime("%m%d")).mean()
        temp_mean.append(estadisticas_mean)

        estadisticas_median = tabla.groupby(tabla.index.strftime("%m%d")).median()
        temp_median.append(estadisticas_median)

        estadisticas_percentil5 = tabla.groupby(tabla.index.strftime("%m%d")).quantile(0.05)
        temp_percentil5.append(estadisticas_percentil5)

        estadisticas_percentil95 = tabla.groupby(tabla.index.strftime("%m%d")).quantile(0.95)
        temp_percentil95.append(estadisticas_percentil95)

    # Agregar las listas temporales a las listas principales
    diario_min.append(temp_min)
    diario_max.append(temp_max)
    diario_mean.append(temp_mean)
    diario_median.append(temp_median)
    diario_percentil5.append(temp_percentil5)
    diario_percentil95.append(temp_percentil95)

* Acomodar los datos y crear tablas con todas las estaciones

In [29]:
# Crear una lista
estadisticos_diarios = [diario_min, diario_max, diario_mean, diario_median, diario_percentil5, diario_percentil95]

* Se selecciona solo la columna que necesitamos, para que la fecha no cause problemas

In [30]:
# Convertir cada tabla en un DataFrame dentro de cada lista
diario_columnas = [[[tabla.iloc[:,0] for tabla in temperatura] for temperatura in estadistico]for estadistico in estadisticos_diarios]

* Volvemos un dataframe cada tabla T(max, min) y luego transponemos cada dataframe

In [31]:
diario_dataframes = [[pd.DataFrame(tabla) for tabla in estadistico]for estadistico in diario_columnas]

In [32]:
diario_dataframes_transpuestos = [[tabla.transpose() for tabla in estadistico]for estadistico in diario_dataframes]

* Renombrar las columnas

In [33]:
# Cambiar los nombres de las columnas en cada DataFrame
for estadistico in diario_dataframes_transpuestos:
    for tabla in estadistico:
        tabla.columns = NUMERO

* Guardar estadisticos diarios en excel

In [34]:
# Crear el objeto ExcelWriter
writer = pd.ExcelWriter('Climatologia_Diaria_Observada_Temperatura.xlsx', engine='xlsxwriter')

# Escribir los DataFrames en diferentes hojas
diario_dataframes_transpuestos[0][0].to_excel(writer, sheet_name='tmax_min', index=True)
diario_dataframes_transpuestos[1][0].to_excel(writer, sheet_name='tmax_max', index=True)
diario_dataframes_transpuestos[2][0].to_excel(writer, sheet_name='tmax_mean', index=True)
diario_dataframes_transpuestos[3][0].to_excel(writer, sheet_name='tmax_median', index=True)
diario_dataframes_transpuestos[4][0].to_excel(writer, sheet_name='tmax_p5', index=True)
diario_dataframes_transpuestos[5][0].to_excel(writer, sheet_name='tmax_p95', index=True)
diario_dataframes_transpuestos[0][1].to_excel(writer, sheet_name='tmin_min', index=True)
diario_dataframes_transpuestos[1][1].to_excel(writer, sheet_name='tmin_max', index=True)
diario_dataframes_transpuestos[2][1].to_excel(writer, sheet_name='tmin_mean', index=True)
diario_dataframes_transpuestos[3][1].to_excel(writer, sheet_name='tmin_median', index=True)
diario_dataframes_transpuestos[4][1].to_excel(writer, sheet_name='tmin_p5', index=True)
diario_dataframes_transpuestos[5][1].to_excel(writer, sheet_name='tmin_p95', index=True)

# Guardar el archivo Excel
writer.save()

## Obtener los estadisticos mensual

In [35]:
# Crear listas para almacenar los resultados de cada estadístico
mensual_min = []
mensual_max = []
mensual_mean = []
mensual_median = []
mensual_percentil5 = []
mensual_percentil95 = []

# Iterar sobre cada lista de tablas en lista_temp
for lista in lista_temp:
    # Crear listas temporales para almacenar los resultados de cada estadístico en la lista actual
    temp_min = []
    temp_max = []
    temp_mean = []
    temp_median = []
    temp_percentil5 = []
    temp_percentil95 = []

    # Iterar sobre cada tabla en la lista actual
    for tabla in lista:
        # Agrupar por mes y día y calcular los estadísticos
        estadisticas_min = tabla.groupby(tabla.index.strftime("%m")).min()
        temp_min.append(estadisticas_min)

        estadisticas_max = tabla.groupby(tabla.index.strftime("%m")).max()
        temp_max.append(estadisticas_max)

        estadisticas_mean = tabla.groupby(tabla.index.strftime("%m")).mean()
        temp_mean.append(estadisticas_mean)

        estadisticas_median = tabla.groupby(tabla.index.strftime("%m")).median()
        temp_median.append(estadisticas_median)

        estadisticas_percentil5 = tabla.groupby(tabla.index.strftime("%m")).quantile(0.05)
        temp_percentil5.append(estadisticas_percentil5)

        estadisticas_percentil95 = tabla.groupby(tabla.index.strftime("%m")).quantile(0.95)
        temp_percentil95.append(estadisticas_percentil95)

    # Agregar las listas temporales a las listas principales
    mensual_min.append(temp_min)
    mensual_max.append(temp_max)
    mensual_mean.append(temp_mean)
    mensual_median.append(temp_median)
    mensual_percentil5.append(temp_percentil5)
    mensual_percentil95.append(temp_percentil95)

In [36]:
# Crear una lista
estadisticos_mensual = [mensual_min, mensual_max, mensual_mean, mensual_median, mensual_percentil5, mensual_percentil95]

# Convertir cada tabla en un DataFrame dentro de cada lista
mensual_columnas = [[[tabla.iloc[:,0] for tabla in temperatura] for temperatura in estadistico]for estadistico in estadisticos_mensual]

# Volvemos un dataframe cada tabla T(max, min) y luego transponemos cada dataframe
mensual_dataframes = [[pd.DataFrame(tabla) for tabla in estadistico]for estadistico in mensual_columnas]
mensual_dataframes_transpuestos = [[tabla.transpose() for tabla in estadistico]for estadistico in mensual_dataframes]

* Renombrar las columnas

In [37]:
# Cambiar los nombres de las columnas en cada DataFrame
for estadistico in mensual_dataframes_transpuestos:
    for tabla in estadistico:
        tabla.columns = NUMERO

* Guardar estadisticos mensuales en excel

In [38]:
# Crear el objeto ExcelWriter
writer = pd.ExcelWriter('Climatologia_Mensual_Observada_Temperatura.xlsx', engine='xlsxwriter')

# Escribir los DataFrames en diferentes hojas
mensual_dataframes_transpuestos[0][0].to_excel(writer, sheet_name='tmax_min', index=True)
mensual_dataframes_transpuestos[1][0].to_excel(writer, sheet_name='tmax_max', index=True)
mensual_dataframes_transpuestos[2][0].to_excel(writer, sheet_name='tmax_mean', index=True)
mensual_dataframes_transpuestos[3][0].to_excel(writer, sheet_name='tmax_median', index=True)
mensual_dataframes_transpuestos[4][0].to_excel(writer, sheet_name='tmax_p5', index=True)
mensual_dataframes_transpuestos[5][0].to_excel(writer, sheet_name='tmax_p95', index=True)
mensual_dataframes_transpuestos[0][1].to_excel(writer, sheet_name='tmin_min', index=True)
mensual_dataframes_transpuestos[1][1].to_excel(writer, sheet_name='tmin_max', index=True)
mensual_dataframes_transpuestos[2][1].to_excel(writer, sheet_name='tmin_mean', index=True)
mensual_dataframes_transpuestos[3][1].to_excel(writer, sheet_name='tmin_median', index=True)
mensual_dataframes_transpuestos[4][1].to_excel(writer, sheet_name='tmin_p5', index=True)
mensual_dataframes_transpuestos[5][1].to_excel(writer, sheet_name='tmin_p95', index=True)

# Guardar el archivo Excel
writer.save()